In [86]:
# Импортируем необходимы библиотеки
import pandas as pd
import json
from datetime import datetime, timedelta

In [87]:
# Загружаем данные из файла data.csv
df = pd.read_csv('/Users/erikbagdasarov/Downloads/telegram/DE Assessment/data.csv')

In [109]:
# Удаляем пустые значения из столбца 'contracts', чтобы избавиться от строк с пропущенными данными
non_empty_contracts = df['contracts'].dropna()

# Преобразуем строковые значения в формате JSON в словари Python
parsed_contracts = non_empty_contracts.apply(json.loads)

# Преобразуется в нормализованную таблицу, где каждая ключ-значение пара становится столбцом
normalized_contracts = pd.json_normalize(parsed_contracts.explode())

# Объединяем идентификатор 'id' из исходного DataFrame с таблицей нормализованных контрактов
df2 = pd.concat([df['id'], normalized_contracts], axis=1)

In [110]:
# Задаем тип данных datetime для claim_date
df2['claim_date'] = pd.to_datetime(df2['claim_date'], dayfirst=True)

### Task 1:
#### Number of claims for last 180 days


In [111]:
df2_friltered = df2.dropna(subset=['claim_date'])

# Проверка на наличие данных после удаления null
if df2_friltered.empty:
    claim_count = -3
else:
    # Находим максимальную дату
    max_date = df2_friltered['claim_date'].max()

    # Задаем пороговое значение в 180 дней
    threshold_date = max_date - timedelta(days=180)

    # Фильтрумем строки за последние 180 дней
    recent_claims = df2_friltered[df2_friltered['claim_date'] >= threshold_date]

    # Считаем уникальные claim_id за 180 дней
    count_claim_id = recent_claims['claim_id'].nunique() if not recent_claims.empty else -3

# Выводим результат
print(f'Number of claims for last 180 days is: {count_claim_id}')

Number of claims for last 180 days is: 1029


### Task 2
#### Sum of exposue of loans without TBC loans.

In [112]:
# Привести loan_summa к числовому типу (например, float) для корректной работы с суммированием
df2['loan_summa'] = pd.to_numeric(df2['loan_summa'], errors='coerce').fillna(0)

# Фильтруем кредиты, чтобы оставить только те, где поле "bank" не в ['LIZ', 'LOM', 'MKO', 'SUG', None]
filtered_loans = df2[~df2['bank'].isin(['LIZ', 'LOM', 'MKO', 'SUG', None])]

# Рассматриваем только кредиты, где contract_date не равен null
non_null_contracts = filtered_loans.dropna(subset=['contract_date'])

# Проверяем на наличие выданных кредитов
if non_null_contracts.empty:
    # Если кредитов вообще нет, возвращаем -1
    loan_exposure = -1
else:
    # Суммируем поле "loan_summa" для выданных кредитов
    loan_exposure_sum = non_null_contracts['loan_summa'].sum()

    # Логика возвращения результата
    loan_exposure = loan_exposure_sum if loan_exposure_sum > 0 else -3

# Вывод результата
print(f'Loan exposure without TBC loans: {loan_exposure}')

Loan exposure without TBC loans: 788028323445.0


### Task 3
#### Number of days since last loan

In [114]:
# Задаем тип данных datetime для contract_date
df2['contract_date'] = pd.to_datetime(df2['contract_date'], errors='coerce')

# Приводим столбец summa к числовому типу
df2['summa'] = pd.to_numeric(df2['summa'], errors='coerce').fillna(0)

# Фильтруем данные, оставляя записи с не нулевой суммой
non_zero_credits = df2[df2['summa'] != 0]

# Проверяем если запись с ненулевой суммой
if non_zero_credits.empty:
    # Если таких записей нет, то возвращаем значение -3
    days_since_last_credit = -3
else:
    # Находим максимальную (последнюю) дату контракта
    last_credit_date = non_zero_credits['claim_date'].max()

    # Проверяем не пуста ли дата
    if pd.isna(last_credit_date):
        # Если дата пуста, то возвращаяем -3
        days_since_last_credit = -1

    else:
        # Рассчитываем кол-во дней от даты до текущего application date
        current_date = datetime.now()
        delta = current_date - last_credit_date
        days_since_last_credit = delta.days

# Выводим результат
print(f'Days since last credit: {days_since_last_credit} days')

Days since last credit: 339 days


### Сохраняем данные в файл contract_features.csv

In [115]:
data = {
    'count_claim_id': [count_claim_id],
    'loan_exposure': [loan_exposure],
    'days_since_last_credit': [days_since_last_credit]
}

df_features = pd.DataFrame(data)

df_features.to_csv('payme_ts/contract_features.csv', index=False)
